- In previous code we just setted Environment Variables as over LangSmith console started seeing traces under the defied project.

- Here now we have a new project so one way to create a separate project for LangSmith is by adjusting the Environment Variables but instead that we will create/update it though the code.

- Also, we will use our `custom_configurations` to add topics,metadata,run_name,etcs

In [9]:
# Libraries Installation
# pip3 install langchain_google_genai --quiet --exists-action i --no-input
# pip3 install langchain --quiet --exists-action i --no-input

# Authenticate User
from google.colab import auth
auth.authenticate_user()

# Import libraries
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tracers.langchain import LangChainTracer # <-- Import the specific Tracer class

# 1. Configure Environment Variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Sequential LLM App" # setting up newer one Tracing projects, however sometime older project may get considered as environment like Google Colab stores Variables at initial run and may ignore in next run so we can explictly pass it during invoking process
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

# 2. DEFINE MODEL
# model1 for generation
model1 = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    project=userdata.get("GOOGLE_CLOUD_PROJECT"),
    location="global",
    temperature=1.3,
    vertexai=True
)

# model2 for summarization
model2 = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    project=userdata.get("GOOGLE_CLOUD_PROJECT"),
    location="global",
    temperature=1.3,
    vertexai=True
)


# PROMPT, PARSER and CHAIN
prompt1 = PromptTemplate.from_template("Generate a detailed report on {topic}")
prompt2 = PromptTemplate.from_template("Generate a 5 pointer summary from the following text \n {text}")

parser = StrOutputParser()

chain = prompt1 | model1 | parser | prompt2 | model2 | parser


# Create a tracer for the specific project
tracer = LangChainTracer(project_name="Sequential LLM App")

# Create a custom configurations
custom_configurations = {
    "callbacks": [tracer],  # <--- This forces the LangSmith project name
    "run_name":"sequential_chain", # <--- This will give a custom name to run_name instead of default which is "RunnableSequence"
    'tags': ['report_generation', 'summarization'],
    'metadata': {
        'model1': 'gemini-2.5-flash',
        'model2': 'gemini-2.5-flash-lite',
        'environment': 'colab_prod'
    }
}

result = chain.invoke({'topic': 'Unemployment in India'},config=custom_configurations)

print(result)

Here's a 5-pointer summary of the provided text on unemployment in India:

*   **Persistent Challenge & Measurement:** India faces a significant unemployment challenge despite its young workforce and economic growth. Data is officially collected by the NSO through the PLFS, while private entities like CMIE provide more frequent updates, with varying figures often seen.

*   **Key Trends and Disparities:** Unemployment rates fluctuate, with recent PLFS data showing a decline. Significant disparities exist across urban vs. rural areas, a concerningly low female labor force participation rate, and higher unemployment among educated youth. The informal sector dominates employment, often with poor working conditions and underemployment.

*   **Root Causes:** Major causes include rapid population growth, inadequate skill development leading to a mismatch with industry needs, slow growth in manufacturing, and the dominance of the informal sector. Poor infrastructure, low agricultural producti